In [1]:
import re 
import biomart as bt
import pandas as ps
import csv

In [220]:
cnaDf = ps.read_csv("/home/afollette/Finder_Data_Repositories/pdxfinder-data-release-head/data_quarantine/UPDOG/UOC-BC/cna/UOC-BC_cna.tsv", sep='\t')
cnaDf[0:0].to_csv("UOC-BC_cna.expanded.tsv", sep='\t', index=False)
cnaDf[0:0].to_csv("UOC-BC_cna.dense.tsv", sep='\t', index=False)
cnaDf.drop_duplicates(inplace=True)
cnaDf.sort_values(by=["chromosome","seq_start_position","seq_end_position"], inplace=True)
genomicCoor = cnaDf["chromosome"].astype(str) + ":" + cnaDf["seq_start_position"].astype(str) + ":" + cnaDf["seq_end_position"].astype(str) + ":1"
uniqGenomicCoor = genomicCoor.drop_duplicates()
coorSize = uniqGenomicCoor.size

In [24]:
server = bt.BiomartServer( "http://ensembl.org/biomart" )
server.is_alive
gene = server.datasets["hsapiens_gene_ensembl"]

In [25]:
uniqGenomicCoor= ps.read_csv("missingChromoPos").values
coorSize = uniqGenomicCoor.size

In [26]:
for i in range(50,coorSize,50):  
    print("FETCHING gene info for set {} ".format(i))
    response = gene.search({ 
            'filters' : {
                'chromosomal_region' : uniqGenomicCoor[i-49:i].tolist()
            },
            'attributes': [
               'hgnc_symbol', 'chromosome_name', 'start_position', 'end_position' 
            ]
            })
    if response.ok:
        geneResponses = response.content.decode()
        geneResponseList = geneResponses.replace('\n', '\t').split('\t')
        with open('symbolByGeneRegion_hgu133plus2_GRCH38.tsv', 'a+') as symbolByGeneRegion:
            for j in range(4, len(geneResponseList)-4, 4):
                writer = csv.writer(symbolByGeneRegion, quoting=csv.QUOTE_NONE)
                row = "{}\t{}\t{}\t{}".format(geneResponseList[j],geneResponseList[j+1], geneResponseList[j+2],geneResponseList[j+3])
                writer.writerow([row])
    else:
        print(response)

FETCHING gene info for set 50 
FETCHING gene info for set 100 
FETCHING gene info for set 150 
FETCHING gene info for set 200 
FETCHING gene info for set 250 
FETCHING gene info for set 300 
FETCHING gene info for set 350 
FETCHING gene info for set 400 
FETCHING gene info for set 450 
FETCHING gene info for set 500 
FETCHING gene info for set 550 
FETCHING gene info for set 600 
FETCHING gene info for set 650 
FETCHING gene info for set 700 
FETCHING gene info for set 750 
FETCHING gene info for set 800 
FETCHING gene info for set 850 
FETCHING gene info for set 900 
FETCHING gene info for set 950 
FETCHING gene info for set 1000 
FETCHING gene info for set 1050 
FETCHING gene info for set 1100 
FETCHING gene info for set 1150 
FETCHING gene info for set 1200 
FETCHING gene info for set 1250 
FETCHING gene info for set 1300 
FETCHING gene info for set 1350 
FETCHING gene info for set 1400 
FETCHING gene info for set 1450 
FETCHING gene info for set 1500 
FETCHING gene info for set 155

In [221]:
symbolResults = ps.read_csv("./symbolByGeneRegion.tsv", sep='\t', names=['symbol','chromosome', 'start','end', 'region'], dtype={'symbol':str,'chromosome':str,'start':int,'end':int, 'region':str})
symbolLookup = symbolResults[~symbolResults['symbol'].isna()]

In [222]:
def expandedDataFrameForEachSymbol(row):
    chromo = row['chromosome']
    lowerBound = row['seq_start_position']
    upperBound = row['seq_end_position']
    chromoMatch = symbolLookup['chromosome'] == chromo
    lowerLimit = symbolLookup['start'] >= lowerBound
    upperLimit = symbolLookup['end'] <= upperBound
    regionMatches = symbolLookup[chromoMatch & lowerLimit & upperLimit].reset_index()
    matchLen = len(regionMatches)
    if matchLen == 0:
        matchLen = 1
    expandedDf = ps.DataFrame()
    expandedDf = ps.concat([ps.DataFrame(row).transpose()]*matchLen, ignore_index=True)
    expandedDf['symbol'] = regionMatches['symbol']
    expandedDf.to_csv('UOC-BC_cna.expanded.tsv',mode='a', header=False, sep='\t', index=False)

In [223]:
def denseDataFrameForEachSymbol(row):
    chromo = row['chromosome']
    lowerBound = row['seq_start_position']
    upperBound = row['seq_end_position']
    chromoMatch = symbolLookup['chromosome'] == chromo
    lowerLimit = symbolLookup['start'] >= lowerBound
    upperLimit = symbolLookup['end'] <= upperBound
    regionMatches = symbolLookup[chromoMatch & lowerLimit & upperLimit].reset_index()
    row['symbol'] = ', '.join(regionMatches['symbol'].values.tolist())
    ps.DataFrame(row).transpose().to_csv('UOC-BC_cna.dense.tsv',mode='a', header=False, sep='\t', index=False)

In [224]:
cnaDf.apply(lambda x : denseDataFrameForEachSymbol(x), axis=1)

439     None
0       None
1034    None
1247    None
1406    None
        ... 
904     None
2047    None
2246    None
2487    None
2553    None
Length: 2554, dtype: object

In [225]:
cnaDf.apply(lambda x : expandedDataFrameForEachSymbol(x), axis=1)

439     None
0       None
1034    None
1247    None
1406    None
        ... 
904     None
2047    None
2246    None
2487    None
2553    None
Length: 2554, dtype: object